In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from mermaid import Mermaid
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from src.prompts.planner_model import StepType

from src.graph.types import State
from src.graph.nodes import (
    coordinator_node,
    planner_node,
    reporter_node,
    research_team_node,
    researcher_node,
    coder_node,
    human_feedback_node,
    background_investigation_node,
)

In [11]:
import logging
logger = logging.getLogger("node execution")

In [2]:
def continue_to_running_research_team(state: State):
    current_plan = state.get("current_plan")
    if not current_plan or not current_plan.steps:
        return "planner"
    if all(step.execution_res for step in current_plan.steps):
        return "planner"
    for step in current_plan.steps:
        if not step.execution_res:
            break
    if step.step_type and step.step_type == StepType.RESEARCH:
        return "researcher"
    if step.step_type and step.step_type == StepType.PROCESSING:
        return "coder"
    return "planner"

In [3]:
def _build_base_graph():
    """Build and return the base state graph with all nodes and edges."""
    builder = StateGraph(State)
    builder.add_edge(START, "coordinator")
    builder.add_node("coordinator", coordinator_node)
    builder.add_node("background_investigator", background_investigation_node)
    builder.add_node("planner", planner_node)
    builder.add_node("reporter", reporter_node)
    builder.add_node("research_team", research_team_node)
    builder.add_node("researcher", researcher_node)
    builder.add_node("coder", coder_node)
    builder.add_node("human_feedback", human_feedback_node)
    builder.add_edge("background_investigator", "planner")
    builder.add_conditional_edges(
        "research_team",
        continue_to_running_research_team,
        ["planner", "researcher", "coder"],
    )
    builder.add_edge("reporter", END)
    return builder

In [4]:
memory = MemorySaver()
def build_graph():
    """Build and return the agent workflow graph without memory."""
    # build state graph
    builder = _build_base_graph()
    return builder.compile()

In [5]:
graph = build_graph()

In [6]:
Mermaid(graph.get_graph(xray=True).draw_mermaid())

In [7]:
question = """What is the outlook for the cloud computing industry both in the west and emerging markets or regions?
What are some of the drivers of its growth and possible challenges or constraints?  What does the competitive landscape look like for leading
providers of cloud services?"""

In [8]:
initial_state = {
        # Runtime Variables
        "messages": [{"role": "user", "content": question}],
        "auto_accepted_plan": True,
        "enable_background_investigation": True,
    }

config = {
        "configurable": {
            "thread_id": "default",
            "max_plan_iterations": 1,
            "max_step_num": 3,
            "mcp_settings": {
                "servers": {
                    "mcp-github-trending": {
                        "transport": "stdio",
                        "command": "uvx",
                        "args": ["mcp-github-trending"],
                        "enabled_tools": ["get_github_trending_repositories"],
                        "add_to_agents": ["researcher"],
                    }
                }
            },
        },
        "recursion_limit": 100,
    }

In [9]:
last_message_cnt = 0
async for s in graph.astream(
    input=initial_state, config=config, stream_mode="values"
):
    try:
        if isinstance(s, dict) and "messages" in s:
            if len(s["messages"]) <= last_message_cnt:
                continue
            last_message_cnt = len(s["messages"])
            message = s["messages"][-1]
            if isinstance(message, tuple):
                print(message)
            else:
                message.pretty_print()
        else:
            # For any other output format
            print(f"Output: {s}")
    except Exception as e:
        logger.error(f"Error processing stream output: {e}")
        print(f"Error processing output: {str(e)}")

logger.info("Async workflow completed successfully")

================================ Human Message =================================

What is the outlook for the cloud computing industry both in the west and emerging markets or regions?
What are some of the drivers of its growth and possible challenges or constraints?  What does the competitive landscape look like for leading
providers of cloud services?
sync [
  {
    "type": "page",
    "title": "United States Top Cloud Computing Service Market Size, Growth ...",
    "url": "https://www.linkedin.com/pulse/united-states-top-cloud-computing-service-market-size-smome/",
    "content": "The United States Top Cloud Computing Service Market: Regional Dynamics and Forecast Insights provides an in-depth examination of market performance across key U.S. regions, including the Northeast, Midwest, South, and West. This analysis highlights regional variations in demand, consumer behavior, competitive landscape, and regulatory influences that shape the Top Cloud Computing Service industry. By expl

NameError: name 'logger' is not defined

In [10]:
#graph steps and message collection
for state in graph.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)

Num Messages:  5 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  5 Next:  ('reporter',)
--------------------------------------------------------------------------------
Num Messages:  5 Next:  ('planner',)
--------------------------------------------------------------------------------
Num Messages:  5 Next:  ('research_team',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ('researcher',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ('research_team',)
--------------------------------------------------------------------------------
Num Messages:  3 Next:  ('researcher',)
--------------------------------------------------------------------------------
Num Messages:  3 Next:  ('research_team',)
--------------------------------------------------------------------------------
Num Messages:  2 Next:  ('researcher',)
-